In [53]:
# CO2 Emissions Prediction Model
# This notebook builds a machine learning model to predict CO2 emissions per capita
# and simulates the impact of GDP growth scenarios on emissions

import pandas as pd 
import numpy as np
import os
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import itertools
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Set up workspace path - go up one directory from scripts folder to project root
workspace_root = os.path.dirname(os.getcwd())

# Set the data directory path
data_dir = os.path.join(workspace_root, 'data')

csv_file_path = os.path.join(data_dir, 'important_variables.csv')

df = pd.read_csv(csv_file_path)


In [54]:
# Prepare feature list by removing target variables and identifiers
# Target: WB_WDI_EN_GHG_ALL_PC_CE_AR5 (total GHG emissions per capita)
# Also exclude CO2 per capita to avoid direct correlation with target
features = list(df.columns)
features_to_remove = ['YEAR' ,'REF_AREA', 'REF_AREA_LABEL','WB_WDI_EN_GHG_ALL_PC_CE_AR5','WB_WDI_EN_GHG_CO2_PC_CE_AR5']
features = [col for col in df.columns if col not in features_to_remove]


#### Let's try Lasso Regression to evaluate variables overall

In [55]:
# Feature Selection using Lasso Regression
# Lasso automatically selects the most important variables by setting some coefficients to zero
# This helps identify which variables are most predictive of CO2 emissions

# Filter for one year to test variable importance
base_year = 2024
df_year = df[df['YEAR'] == base_year].copy()

# Drop rows where the target or any feature is NaN
df_year = df_year.dropna(subset=features)

# Prepare X and y
X_year = df_year[features]
y_year = df_year['WB_WDI_EN_GHG_CO2_PC_CE_AR5']

# Standardize features, required for Lasso
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_year)

# Run Lasso with cross-validation to find best alpha
lasso = LassoCV(cv=5, random_state=42)
lasso.fit(X_scaled, y_year)

# Get non-zero coefficients (selected variables)
coef = pd.Series(lasso.coef_, index=features)
important_features = coef[coef != 0].sort_values(ascending=False)

print(important_features)

# Results from 2020

# Positive values
# 'WB_WDI_EG_USE_PCAP_KG_OE', # Energy use (kg of oil equivalent per capita)
# 'WB_WDI_EG_USE_COMM_FO_ZS',	#Fossil fuel energy consumption (% of total)
# 'WB_WDI_EN_GHG_N2O_ZG_AR5',	#Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)
# 'WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD',	#Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
# 'WB_WDI_EG_ELC_COAL_ZS',	# Electricity production from coal sources (% of total)
# 'WB_WDI_SP_POP_GROW',	# Population growth (annual %)
# 'WB_WDI_EN_GHG_CH4_FE_MT_CE_AR5', # Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)

# Negative values
# 'WB_WDI_EN_GHG_CH4_ZG_AR5',	# Methane (CH4) emissions (total) excluding LULUCF (% change from 1990)
# 'WB_WDI_EG_FEC_RNEW_ZS', # Renewable energy consumption (% of total final energy consumption)
# 'WB_WDI_EG_ELC_NUCL_ZS',	# Electricity production from nuclear sources (% of total)
# 'WB_WDI_EG_USE_COMM_CL_ZS',	#Alternative and nuclear energy (% of total energy use)


# Different results from 2024
# WB_WDI_EG_USE_ELEC_KH_PC          1.988552 #Electric power consumption (kWh per capita)
# WB_WDI_EN_GHG_N2O_ZG_AR5          0.537415 #Nitrous oxide (N2O) emissions (total) excluding LULUCF (% change from 1990)
# WB_WDI_NY_GDP_MKTP_KD_ZG          0.073309 #GDP growth (annual %)
# WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5   -0.010431 #	Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)
# WB_WDI_NY_GDP_PCAP_KD            -0.160338 # GDP per capita (constant 2015 US$)
# WB_WDI_EG_ELC_PETR_ZS            -0.182538 # Electricity production from oil sources (% of total)
# WB_WDI_EG_USE_COMM_GD_PP_KD      -0.847402 # #Energy use (kg of oil equivalent) per $1,000 GDP (constant 2021 PPP)



# Without considering year and country. Most of the predictive variables makes sense.
# Most of them are related to energy use, emmisions and GDP. 

WB_WDI_EG_USE_PCAP_KG_OE          3.430826
WB_WDI_EG_USE_ELEC_KH_PC          1.988552
WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD    1.554828
WB_WDI_EN_GHG_N2O_ZG_AR5          0.537415
WB_WDI_SP_POP_GROW                0.308016
WB_WDI_NY_GDP_MKTP_KD_ZG          0.073309
WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5   -0.010431
WB_WDI_EN_GHG_CH4_ZG_AR5         -0.089267
WB_WDI_EG_FEC_RNEW_ZS            -0.111996
WB_WDI_NY_GDP_PCAP_KD            -0.160338
WB_WDI_EG_ELC_PETR_ZS            -0.182538
WB_WDI_EG_USE_COMM_CL_ZS         -0.717815
WB_WDI_EG_USE_COMM_GD_PP_KD      -0.847402
dtype: float64


/opt/anaconda3/envs/class_cs/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5755277724516361, tolerance: 0.4854100489856266
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/class_cs/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5468840555582233, tolerance: 0.45313136063941606
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/class_cs/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.49999928145211925, tolerance: 0.45313136063941606
  model = cd_fast.enet_coordinate_descent_gram(
/opt/anaconda3/envs/class_cs/lib/python3

In [66]:
# Remove highly correlated variables to avoid multicollinearity issues
# This improves model performance when training across multiple years 
features_to_remove.extend([
    'WB_WDI_EN_GHG_CH4_AG_MT_CE_AR5', #	Methane (CH4) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_CH4_BU_MT_CE_AR5', #	Methane (CH4) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_FE_MT_CE_AR5', # Methane (CH4) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_IC_MT_CE_AR5', #	Methane (CH4) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_MT_CE_AR5',	#Methane (CH4) emissions (total) excluding LULUCF (Mt CO2e)
'WB_WDI_EN_GHG_CH4_PI_MT_CE_AR5',	#Methane (CH4) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_TR_MT_CE_AR5',	#Methane (CH4) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CH4_WA_MT_CE_AR5',	#Methane (CH4) emissions from Waste (Mt CO2e)
'WB_WDI_EN_GHG_CO2_BU_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_FE_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IC_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_IP_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_CO2_PI_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_CO2_RT_GDP_KD',	#Carbon intensity of GDP (kg CO2e per constant 2015 US$ of GDP)
'WB_WDI_EN_GHG_CO2_RT_GDP_PP_KD',	#Carbon intensity of GDP (kg CO2e per 2021 PPP $ of GDP)
'WB_WDI_EN_GHG_CO2_TR_MT_CE_AR5',	#Carbon dioxide (CO2) emissions from Transport (Energy) (Mt CO2e)    
'WB_WDI_EN_GHG_N2O_AG_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Agriculture (Mt CO2e)
'WB_WDI_EN_GHG_N2O_BU_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Building (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_FE_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Fugitive Emissions (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IC_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Combustion (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_IP_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Industrial Processes (Mt CO2e)
'WB_WDI_EN_GHG_N2O_PI_MT_CE_AR5',#	Nitrous oxide (N2O) emissions from Power Industry (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_TR_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Transport (Energy) (Mt CO2e)
'WB_WDI_EN_GHG_N2O_WA_MT_CE_AR5',	#Nitrous oxide (N2O) emissions from Waste (Mt CO2e)    
'WB_WDI_NY_GDP_PCAP_KD',
'WB_WDI_NY_GDP_MKTP_KD',
'WB_WDI_NY_GDP_MKTP_KD_ZG',
'WB_WDI_EN_GHG_CO2_MT_CE_AR5'
    ])


In [67]:
# Model Configuration
# Define key variables and parameters for the prediction model
TARGET_COL   = "WB_WDI_EN_GHG_CO2_PC_CE_AR5"  # Target: Total GHG emissions per capita
YEAR_COL     = "YEAR"
COUNTRY_COL  = "REF_AREA_LABEL"
GDP_GROWTH   = "WB_WDI_NY_GDP_MKTP_KD_ZG"     # GDP growth rate (key predictor)
EXCLUDE_COLS = features_to_remove              # Variables to exclude
INCLUDE_YEAR_AS_FEATURE = True                 # Include year as control variable

TRAIN_MAX_YEAR = 2020                          # Train on data up to 2020, test on 2021+



In [68]:

# Model Training Setup
# Test different combinations of variables to find the best model
TARGET_COL   = "WB_WDI_EN_GHG_CO2_PC_CE_AR5"  # Target variable
YEAR_COL     = "YEAR"
COUNTRY_COL  = "REF_AREA_LABEL"
GDP_GROWTH   = "WB_WDI_NY_GDP_MKTP_KD_ZG"     # GDP growth (always included)
EXCLUDE_COLS = features_to_remove  
TRAIN_MAX_YEAR = 2020                          # Train until this year
MAX_K = 3                                      # Maximum additional variables to test

# Get candidate variables (exclude protected columns)
protected = {TARGET_COL, YEAR_COL, COUNTRY_COL, GDP_GROWTH, *EXCLUDE_COLS}
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
candidates = [c for c in numeric_cols if c not in protected]



In [69]:
# Model Training: Test All Variable Combinations
# Try all combinations of 0-3 additional variables (GDP growth always included)
# Use temporal split: train on 1995-2020, test on 2021-2024
# Include country dummy variables to account for country-specific effects

rows = []

for k in range(0, MAX_K + 1):
    for combo in itertools.combinations(candidates, k):
        features = [GDP_GROWTH] + list(combo)  # GDP always included for simulation
        needed = [TARGET_COL, YEAR_COL, COUNTRY_COL] + features

        # Filter columns with NA
        d = df[needed].dropna().copy()
        if d.empty:
            continue

        # Temporal split: train on historical data, test on recent data
        train = d[d[YEAR_COL] <= TRAIN_MAX_YEAR]
        test  = d[d[YEAR_COL] > TRAIN_MAX_YEAR]
        if train.empty or test.empty:
            continue

        # Create country dummy variables
        X_train = pd.get_dummies(train[[COUNTRY_COL] + features], columns=[COUNTRY_COL], drop_first=True)
        X_test  = pd.get_dummies(test[[COUNTRY_COL] + features], columns=[COUNTRY_COL], drop_first=True)

        # Align columns between train and test sets
        X_train, X_test = X_train.align(X_test, join="left", axis=1, fill_value=0)

        y_train = train[TARGET_COL].values
        y_test = test[TARGET_COL].values

        # Train linear regression model
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate performance metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)

        rows.append({
            "k": k,
            "features": features,
            "rmse": rmse,
            "r2": r2,
            "model": model
        })

# Results: Sort by RMSE (ascending) and R² (descending)
results = pd.DataFrame(rows)
results = results.sort_values(by=["rmse", "r2"], ascending=[True, False]).reset_index(drop=True)

# Show top 5 results 
print(results.head(5)[["k", "features", "rmse", "r2"]])

# Select best model
best_model_row = results.iloc[0]
best_model = best_model_row["model"]
best_features = best_model_row["features"]

print(f"Variables of best model: {best_features}")


   k                                           features      rmse        r2
0  3  [WB_WDI_NY_GDP_MKTP_KD_ZG, WB_WDI_EG_ELC_COAL_...  1.359029  0.947596
1  3  [WB_WDI_NY_GDP_MKTP_KD_ZG, WB_WDI_EG_ELC_COAL_...  1.360597  0.947475
2  3  [WB_WDI_NY_GDP_MKTP_KD_ZG, WB_WDI_EG_ELC_COAL_...  1.367863  0.946913
3  3  [WB_WDI_NY_GDP_MKTP_KD_ZG, WB_WDI_EG_ELC_COAL_...  1.368862  0.946211
4  3  [WB_WDI_NY_GDP_MKTP_KD_ZG, WB_WDI_EG_ELC_FOSL_...  1.378538  0.946081
Variables of best model: ['WB_WDI_NY_GDP_MKTP_KD_ZG', 'WB_WDI_EG_ELC_COAL_ZS', 'WB_WDI_EG_USE_PCAP_KG_OE', 'WB_WDI_EG_USE_COMM_FO_ZS']


## Most predictive variables:
- 'WB_WDI_NY_GDP_MKTP_KD_ZG: 	GDP growth (annual %)
- 'WB_WDI_EG_ELC_COAL_ZS:  Electricity production from coal sources (% of total)
- 'WB_WDI_EG_USE_PCAP_KG_OE: Energy use (kg of oil equivalent per capita)
- 'WB_WDI_EG_USE_COMM_FO_ZS: Fossil fuel energy consumption (% of total)

##### R2 value: 0.947596

In [71]:
# Display Model Coefficients
# Show the intercept and coefficients of the best performing model
print("Intercept (beta_0):", best_model.intercept_)
print("Coefficients (betas):", best_model.coef_)

Intercept (beta_0): -2.366822224249444
Coefficients (betas): [ 1.41789720e-02  2.45700258e-02  1.49152756e-03  4.50215614e-02
 -4.84786375e-01  8.91883944e-01 -2.47828169e-01 -6.23659275e-01
  5.97216694e+00  1.67062512e+00 -8.46870647e-01  6.64565825e+00
 -7.11239210e-01  2.70869151e+00  2.95024345e-01  1.52545325e+00
  2.07371927e+00  6.77859951e-01  3.01207358e+00 -8.93271991e-01
 -3.68870645e-01 -2.16369407e+00  6.33202861e-02  5.40477081e+00
  9.59197879e-01  1.24755483e+00  3.13045311e+00  4.33903932e-01
  1.10745611e+00  3.48966544e+00  1.51294915e+00 -2.97965804e-01
 -2.55955220e-01 -7.35276510e-01  1.76207966e+00  1.35676755e+00
  1.47862596e-01  6.45177720e-01  1.98435788e-01 -3.21783355e-01
  1.77544356e+00  2.67286423e+00  1.57476613e+00 -9.22867795e-01
 -5.82932405e-01 -8.74477834e-01  1.87724177e-01  1.39051347e+00
  1.02983017e+00  4.61469067e+00  1.79651577e-01  1.50214663e+00
  3.07459599e+00  8.50272952e-01 -1.33166172e-01  2.14208375e+00
 -1.79697521e-01  1.69765926e

## Scenario Simulation: GDP Growth Impact on CO2 Emissions

In [72]:

# Scenario Configuration
# Simulate the impact of a 10 percentage point increase in GDP growth on CO2 emissions
COUNTRY_COL  = "REF_AREA_LABEL"
YEAR_COL     = "YEAR"
TARGET_COL   = "WB_WDI_EN_GHG_ALL_PC_CE_AR5"
GDP_GROWTH   = "WB_WDI_NY_GDP_MKTP_KD_ZG"   # GDP growth variable from the model

# Select countries for simulation: 10 developed + 10 developing
developed   = ["United States", "Germany", "United Kingdom", "France", "Japan", "Italy", "Spain", "Canada", "Australia", "New Zealand"]
developing  = ["India", "Mexico", "Brazil", "China", "Indonesia", "Turkey", "Argentina", "Chile", "South Africa", "Korea, Rep."]
selected_countries = developed + developing

YEAR_TARGET = 2024            # Year for simulation 



In [73]:
# Prepare Data for Simulation
# Filter data for the target year and selected countries
base_df = df[(df[YEAR_COL] == YEAR_TARGET) & (df[COUNTRY_COL].isin(selected_countries))].copy()

# Verify all model features exist in the dataset
missing_feats = [c for c in best_features if c not in base_df.columns]



In [74]:
# Run Simulation: Before and After GDP Growth Shock
# Create baseline predictions and then simulate 10 percentage point GDP growth increase

# Baseline prediction (current GDP growth rates)
X_base = pd.get_dummies(base_df[[COUNTRY_COL] + best_features], columns=[COUNTRY_COL], drop_first=True)
X_base = X_base.reindex(columns=best_model.feature_names_in_, fill_value=0)
base_df["pred_CO2_base"] = best_model.predict(X_base)

# Create simulation scenario: increase GDP growth by 10 percentage points
sim_df = base_df.copy()
sim_df[GDP_GROWTH] = sim_df[GDP_GROWTH] + 10.0

# Simulated prediction (after GDP growth shock)
X_sim = pd.get_dummies(sim_df[[COUNTRY_COL] + best_features], columns=[COUNTRY_COL], drop_first=True)
X_sim = X_sim.reindex(columns=best_model.feature_names_in_, fill_value=0)
sim_df["pred_CO2_after"] = best_model.predict(X_sim)


In [75]:
# Calculate Impact: Compare Baseline vs. Simulated CO2 Emissions
# Calculate absolute and percentage changes in CO2 emissions due to GDP growth shock
out = sim_df[[COUNTRY_COL, YEAR_COL, GDP_GROWTH]].copy()
out = out.rename(columns={GDP_GROWTH: "gdp_growth_after"})
out["pred_CO2_base"]  = base_df["pred_CO2_base"].values
out["pred_CO2_after"] = sim_df["pred_CO2_after"].values
out["CO2_change_abs"] = out["pred_CO2_after"] - out["pred_CO2_base"]
out["CO2_change_pct"] = np.where(out["pred_CO2_base"] != 0,
                                 100 * out["CO2_change_abs"] / out["pred_CO2_base"],
                                 np.nan)




In [76]:
# Results Analysis: Display Impact by Country and Development Level
# Add baseline GDP growth for comparison
out = out.merge(
    base_df[[COUNTRY_COL, GDP_GROWTH]],
    on=COUNTRY_COL, how="left"
).rename(columns={GDP_GROWTH: "gdp_growth_base"})

# Sort results: developed countries first, then developing
order_map = {c:i for i,c in enumerate(selected_countries)}
out["__order__"] = out[COUNTRY_COL].map(order_map)
out = out.sort_values(["__order__"]).drop(columns="__order__").reset_index(drop=True)

# Display detailed results
cols_show = [COUNTRY_COL, YEAR_COL, "gdp_growth_base", "gdp_growth_after",
             "pred_CO2_base", "pred_CO2_after", "CO2_change_abs", "CO2_change_pct"]
print(out[cols_show].to_string(index=False))

# Summary by development level
out["group"] = out[COUNTRY_COL].apply(lambda x: "Developed" if x in developed else "Developing")
summary = out.groupby("group")[["CO2_change_abs", "CO2_change_pct"]].agg(["mean","median","min","max"])
print("\nSummary by development level (CO2 change):")
print(summary)

REF_AREA_LABEL  YEAR  gdp_growth_base  gdp_growth_after  pred_CO2_base  pred_CO2_after  CO2_change_abs  CO2_change_pct
 United States  2024         2.796190         12.796190      16.014109       16.155898         0.14179        0.885405
       Germany  2024        -0.238527          9.761473       7.838353        7.980143         0.14179        1.808922
United Kingdom  2024         1.100668         11.100668       5.211986        5.353775         0.14179        2.720455
        France  2024         1.166139         11.166139       4.304080        4.445870         0.14179        3.294309
         Japan  2024         0.083699         10.083699       8.688822        8.830612         0.14179        1.631864
         Italy  2024         0.725792         10.725792       5.755904        5.897694         0.14179        2.463379
         Spain  2024         3.150196         13.150196       5.123978        5.265768         0.14179        2.767180
        Canada  2024         1.528081         11

Simulation of an increase of 10% in gpd among developed and developing countries, leaving the other variables in the model without change, shows that developing countries have a bigger increase on C02 emissions.

The porcentual changes goes from 0.864416 to 3.294309 for developed countries and, from 1.179863 to  7.758931 for developing countries. 

This makes sense if consider that other variables in the model are 
- 'WB_WDI_EG_ELC_COAL_ZS:  Electricity production from coal sources (% of total)
- 'WB_WDI_EG_USE_PCAP_KG_OE: Energy use (kg of oil equivalent per capita)
- 'WB_WDI_EG_USE_COMM_FO_ZS: Fossil fuel energy consumption (% of total)


Electricity production from coal  and Fossil fuel energy consumption might variate among developed and developing countries based on their adoption of green technologies for production of energy. 